In [2]:
import fasttext

In [3]:
import pandas as pd

In [96]:
df = pd.read_csv("adbase_indexed_06062023.csv")[["Actual_product","Actual_category","ML_Transcripted_text",	"ML_Derived_category"	,"ML_Derived_product"]]

In [97]:
df = df.dropna()

In [98]:
df = df[["ML_Transcripted_text", "Actual_product"]]

In [99]:
df.columns = ["transcript", "product"]

In [100]:
df["full_text"] = df["transcript"] + " " + df["product"]

In [101]:
import nltk
import re
from nltk.corpus import stopwords
import string

In [ ]:

def clean_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    tokens = text.split()
    
    # Remove stopwords
    # stop_words = set(stopwords.words('english'))
    stop_words = {}
    tokens = [token for token in tokens if token not in stop_words]
    
    # Join the tokens back into a single string
    cleaned_text = ' '.join(tokens)
    text = text.strip()
    return cleaned_text

df["transcript"] = df["transcript"].apply(clean_text)

In [105]:
df["product"] = df["product"].apply(clean_text)

In [106]:
df["full_text"] = df["full_text"].apply(clean_text)

In [107]:
label_mapping = {i:idx for idx, i in enumerate(list(set(df['product'])))}

In [108]:
df["label"] = df["product"].apply(lambda x: [label_mapping[x]])

In [136]:
df = df.reset_index(drop=True)

In [154]:
trn.iloc[0]["transcript"], tst.iloc[0]["product"]

('come on not again seriously amazing i feel stuck here but i cant afford to leave dont bridge your day job at least not yet you cant afford to reskill keep your job and study online with robertson you can get out of a jam and find a meaningful career you love get your diploma without missing a paycheck',
 'robertson college')

In [156]:
trn.head(2)

,transcript,product,full_text,label
415,the,kesimpta medication,the kesimpta medication,[70]
314,fasten your seatbelts your email campaigns are...,tonybet sports betting casino,fasten your seatbelts your email campaigns are...,[222]


In [110]:
from sklearn.model_selection import train_test_split
from xclib.utils.text import BoWFeatures
import numpy as np
from xclib.utils.sparse import ll_to_sparse
from xclib.data import data_utils

In [143]:
trn, tst = train_test_split(df, test_size=0.2)

In [150]:
def get_bow_fex(trn_text, encoding='latin', min_df=3, dtype=np.float32):
    fex = BoWFeatures(encoding=encoding, min_df=min_df, dtype=dtype)
    fex.fit(trn_text)
    return fex

In [151]:
def transform_text(fex, transcript_train, transcript_test, product_train, product_test):
    Xf_train = fex.transform(transcript_train)
    Xf_test = fex.transform(transcript_test)
    yf_train = fex.transform(product_train)
    yf_test = fex.transform(product_test)

    return Xf_train, Xf_test, yf_train, yf_test

def max_feature_index(trn_labels, tst_labels):
    max_ind = max([max(item) for item in trn_labels])
    return max(max_ind, max([max(item) for item in tst_labels]))

In [157]:
def save_feat(trn, test):
    fex = get_bow_fex(trn["full_text"].to_list())
    Xf_train, Xf_test, yf_train, yf_test = transform_text(fex, trn["transcript"].to_list(),
                                                           tst["transcript"].to_list(),
                                                        trn["product"].to_list(),
                                                         tst["product"].to_list()
                                                        )
    trn_labels = trn["label"].to_list()
    tst_labels = tst["label"].to_list()
    
    max_ind = max_feature_index(trn_labels, tst_labels)
    print(max_ind)
    
    trn_labels = ll_to_sparse(
        trn_labels, shape=(len(trn_labels), max_ind))
    tst_labels = ll_to_sparse(
        tst_labels, shape=(len(tst_labels), max_ind))

    data_utils.write_data("train.txt", Xf_train, trn_labels)
    data_utils.write_data("test.txt", Xf_test, tst_labels)
    return fex

In [ ]:
!perl convert_format.pl train.txt new_data/trn_X_Xf.txt new_data/trn_X_Y.txt
!perl convert_format.pl test.txt new_data/tst_X_Xf.txt new_data/tst_X_Y.txt

In [165]:
vocab_list = list(sorted(list(fex.vocabulary_.keys()),  key=lambda x: fex.vocabulary_[x]))

In [168]:
with open("new_data/Xf.txt", "w") as f:
    for item in vocab_list:
        f.write(item + '\n')

In [169]:
import sys
import re
from xclib.embeddings.fasttext_embeddings import load_model, get_vectors
import numpy as np

In [170]:
def preprocess(line):
    line = line.rstrip()
    return re.sub(r" ", "_", line)


def load_vocabulary(infile):
    with open(infile, 'r',encoding='latin-1') as fp:
        vocabulary = fp.readlines()
    return [preprocess(item) for item in vocabulary]


In [171]:
model = load_model('wiki.en.bin')

ValueError: wiki.en.bin cannot be opened for loading!